<a href="https://colab.research.google.com/github/GimiOne/ML-Pet-Projects/blob/main/%D0%9E%D0%B1%D0%BD%D0%B0%D1%80%D1%83%D0%B6%D0%B5%D0%BD%D0%B8%D0%B5%20%D0%B1%D0%BE%D0%BB%D0%B5%D0%B7%D0%BD%D0%B8%20%D0%9F%D0%B0%D1%80%D0%BA%D0%B8%D0%BD%D1%81%D0%BE%D0%BD%D0%B0%20%D1%81%20%D0%BF%D0%BE%D0%BC%D0%BE%D1%89%D1%8C%D1%8E%20XGBoost%20%D0%B8%20%D0%BD%D0%B5%20%D1%82%D0%BE%D0%BB%D1%8C%D0%BA%D0%BE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Задача 1. Обнаружение фальшивых новостей

Загружаем файл с данными новостей fake_news.csv

In [ ]:
!wget https://storage.yandexcloud.net/academy.ai/practica/fake_news.csv

--2025-02-28 17:15:26--  https://storage.yandexcloud.net/academy.ai/practica/fake_news.csv
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30696129 (29M) [text/csv]
Saving to: ‘fake_news.csv’

fake_news.csv       100%[===================>]  29.27M  8.83MB/s    in 5.1s    

2025-02-28 17:15:33 (5.69 MB/s) - ‘fake_news.csv’ saved [30696129/30696129]



Импортируем необходимые библиотеки

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
#Пример вывода данных из файла fake_news.csv
df=pd.read_csv('fake_news.csv')
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [ ]:
# Проверка на наличие null или пропущенных значений
df.isnull().sum()

,0
Unnamed: 0,0
title,0
text,0
label,0


Построим график количества REAL и FAKE новостей. Как правило, чем более равное количество данных из двух классов, тем корректнее будет результат обучения.

In [ ]:
i = df['label'].value_counts()
fig = go.Figure(data=[go.Bar(
            x=['Real','Fake'],
            y=i,
            text=i,
            textposition='auto',
            marker_color=['green', 'red']
        )])

fig.show()

Разделяем данные на обучающую выборку и тестовую. Применяем алгоритм TF-IDF для перобразования текста в численный вид для обучения через PassiveAggressive алгоритм.

In [ ]:
# Разбиваем выборку на 80% обучающую и 20% тестовую.
X_train,X_test,y_train,y_test=train_test_split(df['text'], df.label, test_size=0.2, random_state=7)

# Определяем объект векторизатора TF-IDF. Удаляем английские стоп-слова, такие как 'the', 'a'.
tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.7)

tfidf_train=tfidf_vectorizer.fit_transform(X_train)
tfidf_test=tfidf_vectorizer.transform(X_test)

pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(tfidf_train,y_train)

y_pred=pac.predict(tfidf_test)

# Оценка точности модели
score = accuracy_score(y_test, y_pred)

Вычисляем точность результата

In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred, labels=['FAKE', 'REAL'])
print("Матрица ошибок:")
print(conf_matrix)

Матрица ошибок:
[[590  48]
 [ 42 587]]


In [ ]:
# Визуализация матрицы ошибок
fig = go.Figure(data=go.Heatmap(
    z=conf_matrix,
    x=['FAKE', 'REAL'],
    y=['FAKE', 'REAL'],
    colorscale='Blues',
    text=conf_matrix,
    texttemplate="%{text}",
))
fig.update_layout(title="Матрица ошибок",
                  xaxis_title="Предсказанные метки",
                  yaxis_title="Истинные метки",
                  xaxis={'side': 'bottom'})
fig.show()

In [ ]:
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 92.9%


In [ ]:
# Визуализация точности модели
correct_predictions = score * 100
incorrect_predictions = 100 - correct_predictions

fig = go.Figure(data=[go.Pie(labels=['Правильные', 'Неправильные'],
                             values=[correct_predictions, incorrect_predictions],
                             hole=.3)])
fig.update_layout(title="Точность модели")
fig.show()


## Задача 2. Обнаружение болезни Паркинсона с помощью XGBoost


### Скачиваем датасет UCI ML Parkinsons

In [ ]:
!wget https://storage.yandexcloud.net/academy.ai/practica/parkinsons.data

--2025-02-28 17:32:25--  https://storage.yandexcloud.net/academy.ai/practica/parkinsons.data
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40697 (40K) [application/x-www-form-urlencoded]
Saving to: ‘parkinsons.data’

parkinsons.data     100%[===================>]  39.74K   138KB/s    in 0.3s    

2025-02-28 17:32:27 (138 KB/s) - ‘parkinsons.data’ saved [40697/40697]



### Импортируем библиотеки и загружаем .csv в датафрейм

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import os, sys
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

df = pd.read_csv('parkinsons.data')
df.head()

,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,...,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,phon_R01_S01_4,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,...,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,phon_R01_S01_5,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,...,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


In [ ]:
# df.loc[:, df.columns != 'status'] выбирает все столбцы, кроме 'status'
# .values[:, 1:] преобразует результат в numpy массив и удаляет первый столбец
all_features = df.loc[:, df.columns != 'status'].values[:, 1:]

# Выбираем целевой столбец 'status' как выходные метки
# .values преобразует результат в numpy массив
out_come = df.loc[:, 'status'].values

# Выводим значения целевого столбца 'status' для проверки
print(out_come)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 1
 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0
 0 0 0 0 0 0 0 0 0 0]


In [ ]:
# MinMaxScaler((-1, 1)) масштабирует данные в диапазон [-1, 1]
scaler = MinMaxScaler((-1, 1))

# fit_transform обучает scaler на данных и сразу применяет преобразование
X = scaler.fit_transform(all_features)
y = out_come

# Разделяем данные на обучающую и тестовую выборку
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

# Параметры для перебора
param_grid = {
    'n_estimators': [50, 100, 200, 300, 400],  # Количество деревьев
    'max_depth': [3, 4, 5,6, 7, 8, 9],           # Максимальная глубина дерева
    'learning_rate': [0.01, 0.1, 0.2, 0.3, 0.5] # Скорость обучения
}

xgb_clf = xgb.XGBClassifier(eval_metric='logloss')

# Поиск лучших гиперпараметров
grid_search = GridSearchCV(estimator=xgb_clf, param_grid=param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Лучшие гиперпараметры
print("Лучшие параметры:", grid_search.best_params_)

# Обучение с лучшими параметрами
best_xgb_clf = grid_search.best_estimator_
best_xgb_clf.fit(X_train, y_train)


Лучшие параметры: {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 400}


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=4,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=400,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [ ]:
# Оценка на тестовых данных
accuracy_train = best_xgb_clf.score(X_train, y_train)
accuracy_test = best_xgb_clf.score(X_test, y_test)

accuracy_train = round(accuracy_train *100, 2)
accuracy_test = round(accuracy_test *100, 2)
print("Точность на обучающих данных:", accuracy_train, "%")
print("Точность на тестовых данных:", accuracy_test, "%")




Точность на обучающих данных: 100.0 %
Точность на тестовых данных: 94.87 %


## Визуализация точности

Для обучающей выборки

In [ ]:
error_test = round(100 - accuracy_test, 2)
error_train = round(100 - accuracy_train, 2)
print(error_train)


go.Figure(data=[go.Pie(labels=['Правильные', 'Неправильные'],
                             values=[accuracy_train, error_train],
                             hole=.3)])

0.0


Для тестовой выборки

In [ ]:
#Визуализация

print(accuracy_test, error_test)
go.Figure(data=[go.Pie(labels=['Правильные', 'Неправильные'],
                             values=[accuracy_test, error_test],
                             hole=.3)])

94.87 5.13


## 3. Творческая задача

**Задача создания рекомендательной системы по фильмам. Поиск списка похожих фильмов на основые одного фильма.**

In [ ]:
!wget https://a.anione.fun/dataset/tmdb_5000_movies.csv

--2025-02-28 18:22:26--  https://a.anione.fun/dataset/tmdb_5000_movies.csv
Resolving a.anione.fun (a.anione.fun)... 104.21.96.1, 104.21.64.1, 104.21.32.1, ...
Connecting to a.anione.fun (a.anione.fun)|104.21.96.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5698602 (5.4M) [text/csv]
Saving to: ‘tmdb_5000_movies.csv’

tmdb_5000_movies.cs 100%[===================>]   5.43M  2.51MB/s    in 2.2s    

2025-02-28 18:22:29 (2.51 MB/s) - ‘tmdb_5000_movies.csv’ saved [5698602/5698602]



In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
df = pd.read_csv ('tmdb_5000_movies.csv')

df.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [ ]:
df['overview'].head(5)

,overview
0,"In the 22nd century, a paraplegic Marine is di..."
1,"Captain Barbossa, long believed to be dead, ha..."
2,A cryptic message from Bond’s past sends him o...
3,Following the death of District Attorney Harve...
4,"John Carter is a war-weary, former military ca..."


In [ ]:
import ast

# Функция для извлечения названий жанров
def get_genre_names(genres):
    try:
        genres_list = ast.literal_eval(genres)
        return ' '.join([genre['name'] for genre in genres_list])
    except (ValueError, TypeError):
        return ''


# Функция для извлечения названий ключевых слов
def get_keyword_names(keywords):
    try:
        keywords_list = ast.literal_eval(keywords)
        return ' '.join([keyword['name'] for keyword in keywords_list])
    except (ValueError, TypeError):
        return ''

# Применяем функцию к столбцу 'genres' и 'keywords'
df['genres'] = df['genres'].apply(get_genre_names)
df['keywords'] = df['keywords'].apply(get_keyword_names)

In [ ]:
# Проверка преобразованных столбцов genres и keywords к текстовому виду
df.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6 bri...,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [ ]:
# Определяем объект векторизатора TF-IDF. Удаляем английские стоп-слова, такие как 'the', 'a'.
# Параметр ngram_range=(1, 2) учитывает не только отдельные слова, но и пары слов. С этим параметром сеть программа работает, кажется, лучше.
tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1, 2))

#Заменяем на пустую строку, если в строке пропущенные значения NaN
df['soup'] = df['genres'].fillna('') + ' ' + df['keywords'].fillna('') + ' ' + df['overview'].fillna('')

# Создаем необходимую матрицу TF-IDF, обучая векторизатор на данных и преобразуя их
tfidf_matrix = tfidf.fit_transform(df['soup'])

# Выводим размерность матрицы tfidf_matrix
print(tfidf_matrix.shape)

(4803, 177447)


In [ ]:
# Импортируем linear_kernel для вычисления похожести матриц
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
# cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

indices = pd.Series(df.index, index=df['title']).drop_duplicates()

indices.head()

,0
title,
Avatar,0
Pirates of the Caribbean: At World's End,1
Spectre,2
The Dark Knight Rises,3
John Carter,4


In [ ]:
# Функция, которая принимает название фильма в качестве входных данных и выводит наиболее похожие фильмы
def get_recommendations(title, cosine_sim=cosine_sim):
    # Получаем индекс фильма, соответствующий заданному названию
    try:
        idx = indices[title]  # Находим индекс по названию фильма
    except KeyError:  # Обработка случая, когда название не найдено
        print(f"Фильм '{title}' не найден в данных.")
        return []  # Возвращаем пустой список

    sim_scores = list(enumerate(cosine_sim[idx]))
    # Сортируем фильмы на основе оценок сходства
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:21]  # Берем 20 фильмов

    # Получаем индексы фильмов
    movie_indices = [i[0] for i in sim_scores]
    recommendations = df.iloc[movie_indices]

    # Фильтрация по популярности и рейтингу
    filtered_recommendations = recommendations[
        (recommendations['popularity'] > 10) & (recommendations['vote_average'] > 6)
    ]

    # Возвращаем топ-10 наиболее похожих фильмов
    return pd.DataFrame({
                'title': df['title'].iloc[movie_indices],
                'genres': df['genres'].iloc[movie_indices]
            })


In [ ]:
get_recommendations('Interstellar')

,title,genres
2966,2001: A Space Odyssey,Science Fiction Mystery Adventure
373,Mission to Mars,Science Fiction
4332,Silent Running,Adventure Drama Science Fiction
1531,Moonraker,Action Adventure Thriller Science Fiction
278,Planet of the Apes,Thriller Science Fiction Action Adventure
3730,Cargo,Thriller Mystery Science Fiction
149,Armageddon,Action Thriller Science Fiction Adventure
643,Space Cowboys,Action Adventure Thriller
770,Event Horizon,Horror Science Fiction Mystery
2015,Spaceballs,Comedy Science Fiction


In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import plotly.express as px

# Уменьшение размерности с помощью t-SNE
tsne = TSNE(n_components=2, random_state=42)
reduced_data = tsne.fit_transform(cosine_sim)

In [ ]:
# Создание DataFrame для визуализации
viz_data = pd.DataFrame(reduced_data, columns=['x', 'y'])
viz_data['title'] = df['title']
viz_data['vote_average'] = df['vote_average']

# print(viz_data.head())

# Интерактивная визуализация с Plotly
fig = px.scatter(viz_data, x='x', y='y', hover_name='title',
                 color='vote_average', title='t-SNE Визуализация схожести фильмов',
                 labels={'x': 't-SNE Измерение 1', 'y': 't-SNE Измерение 2'})

fig.update_traces(marker=dict(size=8, opacity=0.7))

# # Отображение графика
fig.show()

## Вывод

### **Что было изучено**

Был получен опыт работы с векторизацией текста, матрицами сходства и методами уменьшения размерности.

Изучены базовые возможности Plotly для создания интерактивных графиков.

В процессе разработки я столкнулся со сложностями подготовки данных, тестировал различные подходы и выборал наиболее подходящие решения.

### **Чего удалось достичь**

1. Была создана система на основе машинного обучения (TfidfVectorizer и cosine similarity), которая эффективно, но не идеально определяет схожесть между фильмами на основе их описаний, жанров и ключевых слов.

2. С помощью методов уменьшения размерности t-SNE и библиотеки Plotly была создана интерактивная визуализация, позволяющая исследовать связи между фильмами в 2D пространстве.

### **В заключение**

Проект демонстрирует, как можно использовать машинное обучение для анализа и визуализации данных о фильмах. Он сочетает в себе технические навыки в области обработки текста, анализа данных и визуализации. Благодаря гибкости и возможностям для дальнейшего развития, этот проект может стать основой для более сложных решений в области рекомендательных систем и анализа данных.

